In [36]:
import numpy as np
from pandas import DataFrame, read_csv, to_numeric
from sklearn import cluster
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import math

Q1: Parametric Classification: Using each of the 64 input features separately as the single input dimension, use parametric classification, assuming that the input is distributed according to a Gaussian. Report the training and test errors for the case of each of the 64 features. Which feature(s) give the best test performance? 

In [37]:

XT = read_csv('/media/anuja/study/Anuja/MS/CS_697A_ML/hw2/optdigits_69_N200.tra', sep=",")



In [38]:
# Calling DataFrame constructor 
df = DataFrame(XT)
print(df)
print(df.shape)
    

     0  0.1  5  14   4  0.2  0.3  0.4  0.5  0.6  ...  0.27  0.28  0.29  4.6  \
0    0    0  0  10  12    0    0    0    0    0  ...     0     0     0    1   
1    0    0  2  14  10    0    0    0    0    1  ...     0     0     0    1   
2    0    0  2  13   5    0    0    0    0    0  ...     0     0     0    4   
3    0    0  2  13   6    0    0    0    0    0  ...     0     0     0    2   
4    0    0  1  12  11    0    0    0    0    0  ...     2     0     0    2   
..  ..  ... ..  ..  ..  ...  ...  ...  ...  ...  ...   ...   ...   ...  ...   
194  0    0  0   8  13   10    5    0    0    0  ...     0     0     0    0   
195  0    0  0   3  10   13   12    4    0    0  ...     0     0     0    0   
196  0    0  0   4  10   13   12    0    0    0  ...     0     0     0    0   
197  0    0  6  14  16    9    0    0    0    5  ...     0     0     0    8   
198  0    0  2  13  16    7    0    0    0    0  ...     0     0     0    2   

     12.1  14.4   7  0.30  0.31  6.1  
0      11   

In [39]:
# we have 64 attributes + 1 classification column
# all features in x train
X_train =  XT.iloc[:,1:64]
print(X_train)
# outcome or label in Y train
Y_train = XT.iloc[:,64]
print(Y_train)


     0.1  5  14   4  0.2  0.3  0.4  0.5  0.6  13  ...  12  0.27  0.28  0.29  \
0      0  0  10  12    0    0    0    0    0   9  ...  14     0     0     0   
1      0  2  14  10    0    0    0    0    1  14  ...  15     0     0     0   
2      0  2  13   5    0    0    0    0    0  11  ...   6     0     0     0   
3      0  2  13   6    0    0    0    0    0  10  ...  13     0     0     0   
4      0  1  12  11    0    0    0    0    0   8  ...  15     2     0     0   
..   ... ..  ..  ..  ...  ...  ...  ...  ...  ..  ...  ..   ...   ...   ...   
194    0  0   8  13   10    5    0    0    0   8  ...   0     0     0     0   
195    0  0   3  10   13   12    4    0    0   3  ...   0     0     0     0   
196    0  0   4  10   13   12    0    0    0   9  ...   0     0     0     0   
197    0  6  14  16    9    0    0    0    5  15  ...   9     0     0     0   
198    0  2  13  16    7    0    0    0    0   7  ...  10     0     0     0   

     4.6  12.1  14.4   7  0.30  0.31  
0      1    

In [40]:

[rows, cols] = df.shape
# separate the two classes
class1_data = np.array(df.shape)
class2_data = []
for i in range(rows):
    p = XT.iloc[i,64]
    if p == 6:
        np.append(class1_data,XT.iloc[i,:],axis=0)
    if XT.iloc[i,64] == 9:
        class2_data.append(XT.iloc[i,:])
        
print("printing class1 data")        
print(class1_data)
# print(class2_data)



printing class1 data
[199  65]


Q2: Use all the 64 features, assume that inputs are 64 dimensional Gaussians, and assume that for each class the covariance matrix is different. Report the training and test confusion matrices and errors. Hint: eliminate features that have variance zero. 

Q3: Repeat Q2, assuming that all the class covariance matrices are the same. 

Q4: Use the first 10 features in Q1 that gave the best test performance and repeat Q2. Compare the test error performance you got to Q2. 